In [ ]:
import os
import random
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm
import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, get_cosine_schedule_with_warmup

### TOkenizer

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("FacebookAI/roberta-base")

print("Special Tokens in Roberta Tokenizer")
special_tokens = tokenizer.special_tokens_map
print(special_tokens)


special_tokens_idx = {}
for token in special_tokens.values():
    special_tokens_idx[token] = tokenizer.encode(token,add_special_tokens = False)[0]

print(f"Special Token Index: {special_tokens_idx}")

all_tokens_idx = list(range(tokenizer.vocab_size))
all_special_tokens_idx = sorted(list(special_tokens_idx.values()))
all_non_special_tokens_idx = [token for token in all_tokens_idx if token not in all_special_tokens_idx]

### Prepare Dataset

In [ ]:
path_to_data = "./data/harry_potter_txt/"

text_files = os.listdit(path_to_data)

all_text = ""
for book in text_files:
    with open(os.path.join(path_to_data,book),"r") as f:
        text = f.readlines()
        text = [line for line in text if "Page" not in line]
        text = " ".join(text).replace("\n", "")
        text = [word for word in text.split(" ") if len(word)>0]
        text = " ".join(text)
        all_text += text
all_text = all_text.split(".") 

all_text_chunked = [".".join(all_text[i:i+5]) for i in range(0,len(all_text),5)]

tokenized_text = [tokenizer.encode(text) for text in all_text_chunked]